# Feature visualisation
Feature visualisation is a technique to pinpoint image positions indicative of neuron activations. These can be across entier layers, channels or individual neurons.
Feature visualisation is here used to gauge the expressiveness of the features in the training image of off the shelve models. If adequate no finetunine is currently necessary and one can procede with the next stage which is a time difference based self-supervision

In [ ]:
import captum
import torch.nn.functional as F
from captum.attr import IntegratedGradients, NoiseTunnel
from captum.attr import visualization as viz
import numpy as np
from matplotlib.colors import LinearSegmentedColormap



input_img = dataset[8][0][None,:] #transform_normalize(transformed_img).unsqueeze(0)
transformed_img = un_norm(input_img)
model.train()
model_raw.train()
#model = torchvision.models.mobilenet_v3_large(weights="DEFAULT")
output = model(input_img)
output = F.softmax(output, dim=1)
prediction_score, pred_label_idx = torch.unsqueeze(output[0][1], 0)[None,:], torch.tensor([[1]], dtype=torch.long)#torch.topk(output, 1)
# Initialize the attribution algorithm with the model
integrated_gradients = IntegratedGradients(model)

# Ask the algorithm to attribute our output target to
#attributions_ig = integrated_gradients.attribute(input_img, target=pred_label_idx, n_steps=21)
# Show the original image for comparison
#_ = viz.visualize_image_attr(None, np.transpose(transformed_img.squeeze().cpu().detach().numpy(), (1,2,0)),
#                      method="original_image", title="Original Image")
default_cmap = LinearSegmentedColormap.from_list('custom red',
                                                 [(0, '#ffffff'),
                                                  (0.25, '#0000ff'),
                                                  (1, '#0000ff')], N=256)
#_ = viz.visualize_image_attr(np.transpose(attributions_ig.squeeze().cpu().detach().numpy(), (1,2,0)),
#                             np.transpose(transformed_img.squeeze().cpu().detach().numpy(), (1,2,0)),
#                             method='heat_map',
#                             cmap=default_cmap,
#                             show_colorbar=True,
#                             sign='positive',
#                             title='Integrated Gradients')
noise_tunnel = NoiseTunnel(integrated_gradients)
attributions_ig_nt = noise_tunnel.attribute(input_img, nt_samples=7, nt_type='smoothgrad_sq', target=pred_label_idx, n_steps=22, nt_samples_batch_size=2)

#_ = viz.visualize_image_attr_multiple(np.transpose(attributions_ig_nt.squeeze().cpu().detach().numpy(), (1,2,0)),
#                                      np.transpose(transformed_img.squeeze().cpu().detach().numpy(), (1,2,0)),
#                                      ["original_image", "heat_map"],
#                                      ["all", "positive"],
#                                      cmap=default_cmap,
#                                      show_colorbar=True ,)

In [ ]:
print(f"IG of Train")
default_cmap = LinearSegmentedColormap.from_list('custom red',
                                                 [(0, '#ffffff'),
                                                  (0.25, '#00ff04'),
                                                  (1, '#00ff04')], N=256)
fig, ax = viz.visualize_image_attr(np.transpose(attributions_ig_nt.squeeze().cpu().detach().numpy(), (1,2,0)),
                                      original_image=np.transpose(transformed_img.squeeze().cpu().detach().numpy(), (1,2,0)),
                                      sign='positive',
                                     method='blended_heat_map',
                                      #["all", "positive"],
                                      cmap=default_cmap,
                                      show_colorbar=True)
fig.savefig('train_hybridhead_locked.png')

In [ ]:
weights.meta['categories'][466]